# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    """
    Sends a direct request to the OpenAI model without maintaining conversation context.
    
    Parameters:
    - prompt (str): The input text to be processed by the model.
    - model (str): The name of the model to use (default: gpt-3.5-turbo).
    - temperature (float): The randomness level of the response (higher values increase creativity).
    
    Returns:
    - (str): The model's response.

    """
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

###########################################################################################

def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    """
    Sends a request to the OpenAI model while maintaining the conversation context.
    
    Parameters:
    - messages (list): A list of message dictionaries containing conversation history.
    - model (str): The name of the model to use (default: gpt-3.5-turbo).
    - temperature (float): The randomness level of the response.
    
    Returns:
    - (str): The model's response.
    """
    response = client.chat.completions.create(
        model=model,
        messages= message,
        temperature=temperature, 
    )
    return response.choices[0].message.content

* get_completion → Works with a single prompt, no conversation history.
* get_completion_from_messages → Works with a list of messages, keeps conversation history.

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, forsooth!


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, I don't have access to personal information about you, including your name.


In [8]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa. How can I assist you further, Isa?


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [9]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [11]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'47d9246d-270c-491c-a0fa-c55a7b86e6d5': {'version…

In [12]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "size": "medium cheese",
    "price": 10.95
  },
  "toppings": [
    {
      "item": "mushrooms",
      "price": 1.50
    }
  ],
  "drinks": [
    {
      "item": "small coke",
      "price": 2.00
    }
  ],
  "sides": [
    {
      "item": "Greek salad",
      "price": 7.25
    }
  ],
  "total price": 23.20
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## Version 1: Classic Pizza OrderBot 🍕

In [42]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, a fast and efficient assistant for a pizza restaurant. 
You take orders quickly, confirm details, and provide prices without extra conversation. 
Keep responses brief and to the point.
             
The menu includes:\
- Pepperoni Pizza: $12.95 (Large), $10.00 (Medium), $7.00 (Small)\
- Cheese Pizza: $10.95 (Large), $9.25 (Medium), $6.50 (Small)\
- Eggplant Pizza: $11.95 (Large), $9.75 (Medium), $6.75 (Small)\
- Fries: $4.50 (Large), $3.50 (Small)\
- Greek Salad: $7.25\
Toppings:\
- Extra Cheese: $2.00\
- Mushrooms: $1.50\
- Sausage: $3.00\
- Canadian Bacon: $3.50\
- AI Sauce: $1.50\
- Peppers: $1.00\
Drinks:\
- Coke: $3.00 (Large), $2.00 (Medium), $1.00 (Small)\
- Sprite: $3.00 (Large), $2.00 (Medium), $1.00 (Small)\
- Bottled Water: $5.00"\
"""
} ]  # accumulate messages\


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=700),
)

dashboard



BokehModel(combine_events=True, render_bundle={'docs_json': {'a92ee0f1-da37-405c-8ee7-a48cbf3359ed': {'version…

In [36]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "order": {
    "pizza": {
      "size": "Medium",
      "type": "Cheese",
      "toppings": ["Extra Cheese", "Mushrooms"],
      "price": 12.75
    },
    "drinks": [
      {
        "type": "Coke",
        "size": "Small",
        "price": 1.00
      }
    ],
    "sides": [
      {
        "item": "Greek Salad",
        "price": 7.25
      }
    ],
    "total price": 20.00
  }
}


## Version 2: Friendly Pizza OrderBot 😃

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role': 'system', 'content': """
You are OrderBot, a fun and engaging pizza ordering assistant! 
Your mission is to make ordering a delightful experience.

### How to interact:
- Start with a warm and friendly greeting!
- Ask the customer what they're craving today.
- Suggest delicious extras like a friend who knows the best combos! 
- Help them choose the right size.
- Confirm the order with excitement.
- Ask about pickup or delivery in a fun way.
- If delivery, collect the address politely.
- Gather payment details smoothly.
- Wrap up with a cheerful farewell! 
             
The menu includes:\
- Pepperoni Pizza: $12.95 (Large), $10.00 (Medium), $7.00 (Small)\
- Cheese Pizza: $10.95 (Large), $9.25 (Medium), $6.50 (Small)\
- Eggplant Pizza: $11.95 (Large), $9.75 (Medium), $6.75 (Small)\
- Fries: $4.50 (Large), $3.50 (Small)\
- Greek Salad: $7.25\
Toppings:\
- Extra Cheese: $2.00\
- Mushrooms: $1.50\
- Sausage: $3.00\
- Canadian Bacon: $3.50\
- AI Sauce: $1.50\
- Peppers: $1.00\
Drinks:\
- Coke: $3.00 (Large), $2.00 (Medium), $1.00 (Small)\
- Sprite: $3.00 (Large), $2.00 (Medium), $1.00 (Small)\
- Bottled Water: $5.00"\
"""
} ]  # accumulate messages\


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=900),
)

dashboard


BokehModel(combine_events=True, render_bundle={'docs_json': {'286f4fea-9ed2-43ed-a087-19b974953fd6': {'version…

In [48]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "pizza": {
        "type": "Cheese",
        "size": "Medium",
        "price": 9.25
    },
    "toppings": [
        {
            "name": "Extra Cheese",
            "price": 2.00
        },
        {
            "name": "Mushrooms",
            "price": 1.50
        }
    ],
    "drinks": [
        {
            "name": "Coke",
            "size": "Small",
            "price": 1.00
        }
    ],
    "sides": [
        {
            "name": "Greek Salad",
            "price": 7.25
        },
        {
            "name": "Creamy Tiramisu",
            "price": 5.00
        }
    ],
    "total price": 25.75
}
```


## Version 3: Smart Pizza OrderBot 🤖

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role': 'system', 'content': """
You are an intelligent OrderBot that remembers previous orders and suggests options accordingly. 
Your goal is to make ordering smooth, efficient, and personalized.

How to interact:
- If the user has ordered before, remind them of their past choices and suggest reordering with a discount.
- Suggest popular items based on trends.
- Keep the conversation concise, smart, and engaging.
- Calculate and display the total order price.
             
The menu includes:\
- Pepperoni Pizza: $12.95 (Large), $10.00 (Medium), $7.00 (Small)\
- Cheese Pizza: $10.95 (Large), $9.25 (Medium), $6.50 (Small)\
- Eggplant Pizza: $11.95 (Large), $9.75 (Medium), $6.75 (Small)\
- Fries: $4.50 (Large), $3.50 (Small)\
- Greek Salad: $7.25\
Toppings:\
- Extra Cheese: $2.00\
- Mushrooms: $1.50\
- Sausage: $3.00\
- Canadian Bacon: $3.50\
- AI Sauce: $1.50\
- Peppers: $1.00\
Drinks:\
- Coke: $3.00 (Large), $2.00 (Medium), $1.00 (Small)\
- Sprite: $3.00 (Large), $2.00 (Medium), $1.00 (Small)\
- Bottled Water: $5.00"\
            
**Special Offer!** Reorder your last meal and get 10% off!"""
            
            } ]  # accumulate messages\




inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=800),
)

dashboard


BokehModel(combine_events=True, render_bundle={'docs_json': {'3fa5dbd9-80dc-4e69-8a0a-3483f7d39d79': {'version…

# 📝 Evaluation of Different OrderBot Versions  





## 📌 Overview  
This report reviews the performance of three unique OrderBot versions, each with a distinct approach to order management. The focus is on efficiency, user engagement, and order accuracy.  

---  

## Version 1: (Quick & Direct)  
### ✅ Pros:  
* Fast and straightforward interaction.
* Provides accurate pricing and options.
* Simple and efficient for users who want to order quickly.
### ❌ Cons:  
* No recommendations based on customer preferences.
* Limited engagement, making the experience less interactive.

---  

## Version 2: (Engaging & Conversational)  
### ✅ Pros:  
* Friendly and welcoming tone enhances user experience.
* Suggests additional items naturally.
* Guides hesitant users toward popular options. 
### ❌ Cons:  
* Can be slightly slower due to extra dialogue.
* Some users may find it too chatty.

---  

## Version 3: (Detailed & Precise)  
* Remembers past orders and suggests them with discounts.
* It confirms the order, increasing clarity and trust.
* It allows the user to add more items before checkout, enhancing flexibility and engagement.

---  

## 🎯 Conclusion & Enhancements  
- A **hybrid of Version 2 & Version 3** could offer both clarity and engagement.  
